In [1]:
import requests

#custom request and preprocessing
from help_response import create_list_of_unigue_answers, cleanhtml, clean_punct, read_xml

In [2]:
#load fast ai model
import torch

from fastai.text.models import AWD_LSTM
from fastai.text.learner import get_language_model

awd_lstm_lm_config_custom = dict(emb_sz=768, n_hid=1152, n_layers=3, pad_token=1, qrnn=False, bidir=False, output_p=0.1, hidden_p=0.15, input_p=0.25, embed_p=0.02, weight_p=0.2, tie_weights=True, out_bias=True)

m = get_language_model(arch=AWD_LSTM, vocab_sz = 60004, config = awd_lstm_lm_config_custom)
# state = torch.load('./wt103/fwd_wt103.h5')

In [12]:
l = m.modules()

[SequentialRNN(
   (0): AWD_LSTM(
     (encoder): Embedding(60004, 768, padding_idx=1)
     (encoder_dp): EmbeddingDropout(
       (emb): Embedding(60004, 768, padding_idx=1)
     )
     (rnns): ModuleList(
       (0): WeightDropout(
         (module): LSTM(768, 1152, batch_first=True)
       )
       (1): WeightDropout(
         (module): LSTM(1152, 1152, batch_first=True)
       )
       (2): WeightDropout(
         (module): LSTM(1152, 768, batch_first=True)
       )
     )
     (input_dp): RNNDropout()
     (hidden_dps): ModuleList(
       (0): RNNDropout()
       (1): RNNDropout()
       (2): RNNDropout()
     )
   )
   (1): LinearDecoder(
     (decoder): Linear(in_features=768, out_features=60004, bias=True)
     (output_dp): RNNDropout()
   )
 ), AWD_LSTM(
   (encoder): Embedding(60004, 768, padding_idx=1)
   (encoder_dp): EmbeddingDropout(
     (emb): Embedding(60004, 768, padding_idx=1)
   )
   (rnns): ModuleList(
     (0): WeightDropout(
       (module): LSTM(768, 1152, batch

In [3]:
## be sure that model has one AWD_LSTM
assert l[1].rnns[0].module.weight_ih_l0 == l[4][0].module.weight_ih_l0

NameError: name 'l' is not defined

In [15]:
import torch
import torchvision.models as Model
from torch import nn

class over_AWD_LSTM(nn.Module):
    def __init__(self, extra_model, emb_size, ulm_fit_emb_size = 400):
        super(over_AWD_LSTM, self).__init__()
        self.modules = [module for module in extra_model.modules()]
        self.rnns = self.modules[0][0].rnns
        self.linear = nn.Linear(emb_size, ulm_fit_emb_size)
    def forward(self, input_tensor, from_embeddings = True):
        #print ("input_tensor shape 0 ", input_tensor.shape)
        #tsr = self.linear(input_tensor)
        #print ("input_tensor shape 1 ", tsr.shape)
        tsr1 = self.rnns[0](input_tensor)
        tsr2 = self.rnns[1](tsr1[0])
        tsr3 = self.rnns[2](tsr2[0]) # output, (h_n, c_n)
        return tsr3 # output, (h_n, c_n)

In [22]:
from bert_serving.client import BertClient
from sklearn.metrics.pairwise import cosine_similarity
bc = BertClient()

import re

def create_tensor(strng):
    #print ((clean_punct(cleanhtml(strng))).split())
    arr = bc.encode((clean_punct(cleanhtml(strng))).split())
    return torch.tensor(np.copy(arr))
    

def make_scores_lstm(my_rnn, query, answers, emb_size = 768, hidden_size = 64):
    query_embs = bc.encode((clean_punct(cleanhtml(query))).split())
    tensor_query_embs = torch.tensor(query_embs)
    
    all_query_hidden, (last_query_hidden, last_query_state) = my_rnn(tensor_query_embs.unsqueeze(0))
    
    #last_answers_hidden = [my_rnn(torch.tensor(bc.encode((clean_punct(cleanhtml(answer))).split())).unsqueeze(0))[1] for answer in answers]
    last_answers_hidden = [my_rnn(create_tensor(answer).unsqueeze(0))[1][0] for answer in answers]
    
    aa = last_query_hidden.detach().numpy()
    #print ("aa shape", aa.shape)
    bb = last_answers_hidden[0].detach().numpy()
    #print ("bb shape", bb.shape)
    
    scores = [cosine_similarity(last_query_hidden.squeeze(0).detach().numpy(), last_answer_hidden.squeeze(0).detach().numpy())[0][0] for last_answer_hidden in last_answers_hidden]
    return scores
    

In [23]:
#numpy
import numpy as np

#torch
import torch, torch.nn as nn
import torch.nn.functional as F

#bert service
from bert_serving.client import BertClient
from sklearn.metrics.pairwise import cosine_similarity
bc = BertClient()

# transformers
import pytorch_transformers
from pytorch_transformers import *



In [24]:
def run_baseline(output_dir = '/notebook/touche/output/', input_dir = '/notebook/touche/', input_file = 'topics-task-2-only-titles.xml'):
    
    #transformer part
    config_class, model_class, tokenizer_class = BertConfig, BertModel, BertTokenizer
    config = config_class.from_pretrained('bert-base-uncased')
    config.output_attentions=True
    model = model_class.from_pretrained('bert-base-uncased', config=config)
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
    
    # load responses for all queries from file 
    my_response_list = create_list_of_unigue_answers(input_dir = input_dir, input_file = input_file)
    
    list_of_tuples = read_xml(input_dir + input_file)
    common_list = []
    
    awd_lstm_lm_config_custom = dict(emb_sz=768, n_hid=1152, n_layers=3, pad_token=1, qrnn=False, bidir=False, output_p=0.1, hidden_p=0.15, input_p=0.25, embed_p=0.02, weight_p=0.2, tie_weights=True, out_bias=True)
    m = get_language_model(arch=AWD_LSTM, vocab_sz = 60004, config = awd_lstm_lm_config_custom)
    
    my_rnn = over_AWD_LSTM(m, 768)
        
    print ("lstm ulm", flush = True)    
    for ind_q, elem in enumerate(list_of_tuples[:5]):
        qid = elem[0]
        Q0 = 'Q0'
        query = elem[1]
        tag = 'ULMFIT_LSTM'
        responses = list(zip(*my_response_list[str(ind_q + 1)]))

        scores0 = responses[0]
        #print ("0")
        docs = responses[1]
        #print ("1")
        titles = responses[2]
        #print ("2")
        answers_bodies = responses[3]
        #print ("3")
        # print (scores0, scores3, scores)
        scores = make_scores_lstm(my_rnn, query, answers_bodies)
        qids = qid*len(scores)
        Q0s = [Q0 for elem in scores]
        queries = query*len(scores)
        tags = [tag for elem in scores]
        part_of_commom_list = list(zip(qids, Q0s, docs, scores, tags))
        part_of_commom_list = sorted(part_of_commom_list, key = lambda x: x[3], reverse = True) 

        qids, Q0s, docs, scores, tags = zip(*part_of_commom_list)

        ranks = range(1, len(scores) + 1)
        part_of_commom_list = list(zip(qids, Q0s, docs, ranks, scores, tags))
        common_list = common_list + part_of_commom_list
        
    

    with open(output_dir + 'run_example_ulm.txt', 'w') as fp:
        fp.write('\n'.join('%s %s %s %s %s %s' % x for x in common_list))

In [25]:
run_baseline()

1
[<DOM Element: topics at 0x7f788c043d70>]
50
sze 10
sze 100
sze 10
sze 100
sze 10
else
sze 10
sze 100
sze 10
sze 100
1
[<DOM Element: topics at 0x7f788c087d70>]
50
lstm ulm
